## Setup

In [1]:
# Get raw advent-of-code data
from aocd.models import Puzzle

puzzle = Puzzle(year=2025, day=12)
input_data = puzzle.input_data
example = puzzle.examples[0]

In [2]:
import sys
from pathlib import Path

sys.path.append(str(Path.cwd().parent))

from common.utils.perf_check import time_solution

## Part a


This looks very hard, but let's try to start simple and begin with identifying:
- regions that can definitely fit: region_area > n_presents * 3 * 3
- regions that can definitely not fit their presents : region area < sum(present.area for present in presents)

Any regions that are left, we need to try to solve som packing problem.

In [24]:
# Functions
def count_packable_regions(input_data: str) -> tuple[int, int, int]:
    """Count the number of regions that can pack it's designated presents."""
    *present_strs, regions_str = input_data.split("\n\n")

    # Parse presents: {idx: (shape, shape_area)}
    presents = {
        int((parts := s.split(":"))[0]): (
            (shape := [[1 if c == "#" else 0 for c in row] for row in parts[1].split("\n")[1:]]),
            sum(sum(r) for r in shape),
        )
        for s in present_strs
    }

    # Parse regions: [((w, h), [present_counts])]
    regions = [
        (tuple(map(int, (parts := line.split(":"))[0].split("x"))), [*map(int, parts[1].split())])
        for line in regions_str.splitlines()
    ]

    total_yes_count = total_no_count = total_maybe_count = 0
    for (w, h), present_counts in regions:
        # Check if the region definitely fits its presents
        if (region_area := w * h) >= sum(present_counts) * 3 * 3:
            total_yes_count += 1
        # Check if the region definitely does not fit its presents
        elif region_area < sum(presents[present_id][1] * count for present_id, count in enumerate(present_counts)):
            total_no_count += 1
        else:
            # We should put the packing logic here later
            total_maybe_count += 1

    return total_yes_count, total_no_count, total_maybe_count

Let's see how many regions are definitely fitting or definitely not fitting.


In [23]:
count_packable_regions(input_data)

(541, 459, 0)

Aha! It turns out that there are no "maybe" regions in the input data!
This makes it very simple.

In [27]:
# Performance check
time_a = time_solution(count_packable_regions, input_data)
print(f"The hacky approach takes {time_a:.1f} ms per run.")

The hacky approach takes 1.0 ms per run.


In [29]:
# Submit answers
puzzle.answer_a = str(count_packable_regions(input_data)[0])

That's the right answer!  You are one gold star closer to decorating the North Pole. [Continue to Part Two]
